In [7]:
'''
****************************** Change Log ******************************
7/13
- Now displays game value as fraction AND decimal
- Now uses itertools to get only combinations (as opposed to all permutations)
'''

import itertools
import numpy as np
import pandas as pd

# *************************** Graph Builders ***************************
def generate_cycle(size:int) -> [[int]]:
    arr = []
    for i in range(size):
        row = []
        for j in range(size):
            if abs(i-j) == 1 or abs(i-j) == size - 1:
                row.append(1)
            else:
                row.append(0)
        arr.append(row)
    return arr

def generate_wheel(size:int) -> [[int]]:
    wheel = generate_cycle(size)
    for i in range(size):
        if i != size-1:
            wheel[i][size-1] = 1
        else:
            wheel[i] = [1 for j in range(size)]
    wheel[size-1][size-1] = 0
    return wheel

def generate_star(size:int) -> [[int]]:
    star = [[1 if i==size-1 else 0 for i in range(size)] if j!=size-1 else 
            [0 if k==size-1 else 1 for k in range(size)] for j in range(size)]
    return star

def hamming_distance(v1:[int],v2:[int])->int:
    distance = 0
    for i in range(len(v1)):
        if v1[i] != v2[i]:
            distance += 1
    return distance

def generate_cube(dimension:int) -> [[int]]:
    vertices = itertools.product(range(2), repeat = dimension)
    vertices = [x for x in vertices]
    graph = []
    for v1 in vertices:
        row = []
        for v2 in vertices:
            if hamming_distance(v1,v2) == 1:
                row.append(1)
            else:
                row.append(0)
        graph.append(row)
    return graph
    
def generate_graph()->([[int]],str):
    print("Enter graph type: ")
    graph_type = input("'C' for cycle, 'W' for wheel, 'S' for star, 'Q' for cube, 'STOP' to stop: ")
    while graph_type.lower() not in ['c','w','s','q','stop']:
        print("Invalid type entered.")
        graph_type = input("'C' for cycle, 'W' for wheel, 'S' for star, 'Q' for cube, 'STOP' to stop: ")
    if graph_type.lower() == 'c':
        length = int(input("Enter cycle size: "))
        return (generate_cycle(length),'c')
    elif graph_type.lower() == 'w':
        length = int(input("Enter wheel size: "))
        return (generate_wheel(length),'w')
    elif graph_type.lower() == 's':
        length = int(input("Enter star size: "))
        return (generate_star(length),'s')
    elif graph_type.lower() == 'q':
        dim = int(input("Enter cube dimension: "))
        return (generate_cube(dim),'q')
    else:
        return ([], '')

def generate_graph_param(graph_type:str, n:int)->[[int]]:
    if graph_type.lower() == 'c':
        return generate_cycle(n)
    elif graph_type.lower() == 'w':
        return generate_wheel(n)
    elif graph_type.lower() == 's':
        return generate_star(n)
    elif graph_type.lower() == 'q':
        return generate_cube(n)

    
def build_identity_comp(vertices:int) -> [int]:
    if vertices == 1:
        return [1]
    else:
        i = []
        for k in range(vertices):
            row = []
            for j in range(vertices):
                if k != j:
                    row.append(1)
                else:
                    row.append(0)
            i.append(row)
    return i

# *************************** Calculations ***************************    
def strat_value(arr:[[int]], strategy:[int]) -> (float,int):
    i = np.array(build_identity_comp(len(arr)))
    arr = np.array(arr)
    complement = np.subtract(i,arr)
    winning_cases = 0
    for q1 in range(len(strategy)):
        for q2 in range(len(strategy)):
            if q1==q2:
                winning_cases += 1
            else:
                winning_cases += complement[strategy[q1]][strategy[q2]]
    return (winning_cases / len(strategy) ** 2, winning_cases)
   
def game_value(graph:[[int]], set_size:int)->([int],float,int,int):
    #strategies = create_strategy_list(chain_len,set_size)
    max_value = 0
    numerator = 0
    denominator = set_size ** 2
    best_strats = []
    for strategy in itertools.combinations_with_replacement(range(len(graph)),set_size):
        value, num = strat_value(graph, strategy)
        if value > max_value:
            max_value = value
            numerator = num
            best_strats = [strategy]
        elif value == max_value:
            best_strats.append(strategy)
    return (best_strats, max_value,numerator,denominator)
    
# ******************************* Main *******************************
def main():
    graph, graph_type = generate_graph()
    while (graph):
        size = int(input("Enter desired size of independent set: "))
        strats, value, num, denom = game_value(graph,size)
        print("Best strategies: ")
        count = 0
        for strategy in strats:
            if graph_type != 'q':
                print(strategy, end = ' ')
            else:
                print([format(x, '03b') for x in strategy], end = ' ')
            if not (count+1) % 5:
                print()
            count += 1
        print()
        print("Value of the game:", value, ", ", num, "/", denom)
        print("----------------------------------------------------------------------------------------")
        graph, graph_type = generate_graph()
    
if __name__ == "__main__":
    pass
    main()
#Progress bar 
#Generate tables for other types of graphs
#Larger k values
#Product games - probably with themselves, but maybe others
    
    

Enter graph type: 


In [9]:
# *************************** CSV Data for Cycles ****************************
def generate_csv(graph_type:str, ns:[int], ks:[int], file_name:str):
    all_values = []
    for k in ks:
        values = []
        for n in ns:
            strats, value, num, denom = game_value(generate_graph_param(graph_type,n),k)
            values.append(value)
        all_values.append(values) 
    df = pd.DataFrame(all_values,columns=ns, index = ks)
    df.to_csv(file_name)

generate_csv('s',[1,2,3],[1,2,3], "test.csv")

In [56]:
'''
******************************* Garbage *******************************
def create_strategy_list(chain_len:int, set_size:int)->[int]:
    strategies = []
    for num in range(chain_len ** set_size):
        strategy = []
        for answer in range(set_size):
            strategy.append(int(num/chain_len**answer)%chain_len)
        if sorted(strategy) not in strategies:
            strategies.append(sorted(strategy))
    return strategies
    
def strat_value_cycle(nums:[int], chain_len:int)->(float, int):
    winning_cases = 0
    for q1 in range(len(nums)):
        for q2 in range(len(nums)):
            difference = abs(nums[q1] - nums[q2])
            if q1==q2:
                winning_cases += 1
            # Since question a,b,c,etc. are the complete graph, they are all adjacent, so 
            # the corresponding answers must be adjacent as well.
            # Since we are mapping the complete graph to the complement graph of a chain, 
            # as long as the answers are one away from each other, they are adjacent.
            elif difference != 1 and difference != 0 and difference != chain_len-1:
                winning_cases += 1
    return (winning_cases / len(nums)**2, winning_cases)
 '''

(0.7777777777777778, 7)

[(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1)]
